In [8]:
import pandas as pd
from datetime import datetime
from numpy import concatenate
from matplotlib import pyplot
import numpy as np

# read in the data
dataset = pd.read_csv('prepared_gasoline.csv')

# read in the geo data
geo = pd.read_csv('latlongeo.csv')
geo.drop(['Unnamed: 0', 'region'], axis=1, inplace=True)

# join them -- inner drops the lat/lon nan values
dataset = dataset.merge(geo, how='inner', on=['latitude', 'longitude'])

dataset.shape

(8110884, 19)

In [9]:
dataset['state'].unique()

array(['North Rhine-Westphalia', 'Bavaria', 'Baden-Wuerttemberg', 'Hesse',
       'Thuringia', 'Rheinland-Pfalz', 'Saxony', 'Schleswig-Holstein',
       'Lower Saxony', 'Mecklenburg-Vorpommern', 'Saxony-Anhalt',
       'Brandenburg', 'Aargau', 'Berlin', 'Hamburg', 'Bremen', 'Saarland',
       'Vorarlberg', 'Wallonia', 'Basel-City', 'Limburg', 'Lorraine',
       'Tyrol', 'Thurgau', 'Schaffhausen', 'Gelderland', 'Drenthe'],
      dtype=object)

In [10]:
dataset.loc[dataset['state'] =='Gelderland', 'state'] = 'North Rhine-Westphalia'
dataset.loc[dataset['state'] =='Drenthe', 'state'] = 'Lower Saxony'
dataset.loc[dataset['state'] =='Schaffhausen', 'state'] = 'Baden-Wuerttemberg'
dataset.loc[dataset['state'] =='Thurgau', 'state'] = 'Baden-Wuerttemberg'
dataset.loc[dataset['state'] =='Tyrol', 'state'] = 'Bavaria'
dataset.loc[dataset['state'] =='Lorraine', 'state'] = 'Saarland'
dataset.loc[dataset['state'] =='Limburg', 'state'] = 'North Rhine-Westphalia'
dataset.loc[dataset['state'] =='Basel-City', 'state'] = 'Baden-Wuerttemberg'
dataset.loc[dataset['state'] =='Wallonia', 'state'] = 'North Rhine-Westphalia'
dataset.loc[dataset['state'] =='Vorarlberg', 'state'] = 'Bavaria'
dataset.loc[dataset['state'] =='Aargau', 'state'] = 'Baden-Wuerttemberg'

In [11]:
dataset['state'].unique()

array(['North Rhine-Westphalia', 'Bavaria', 'Baden-Wuerttemberg', 'Hesse',
       'Thuringia', 'Rheinland-Pfalz', 'Saxony', 'Schleswig-Holstein',
       'Lower Saxony', 'Mecklenburg-Vorpommern', 'Saxony-Anhalt',
       'Brandenburg', 'Berlin', 'Hamburg', 'Bremen', 'Saarland'],
      dtype=object)

In [12]:
test = dataset['state']

In [13]:
# one hot encode state
dataset['state'] = pd.Categorical(dataset['state']).codes

test_num = dataset['state']

# remove e5gas NAs
dataset = dataset[np.isfinite(dataset['e5gas'])]

# remove e5gas 0s
dataset = dataset[dataset['e5gas'] != 0]

In [18]:
test.iloc[1169:1175]

1169               Bavaria
1170    Baden-Wuerttemberg
1171    Baden-Wuerttemberg
1172    Baden-Wuerttemberg
1173    Baden-Wuerttemberg
1174    Baden-Wuerttemberg
Name: state, dtype: object

In [15]:
test_num[test_num == 0]

1170       0
1171       0
1172       0
1173       0
1174       0
1175       0
1176       0
1177       0
1178       0
1179       0
1180       0
1181       0
1182       0
1183       0
1184       0
1185       0
1186       0
1187       0
1188       0
1189       0
1190       0
1191       0
1192       0
1193       0
1194       0
1195       0
1196       0
1197       0
1198       0
1199       0
          ..
8106829    0
8106830    0
8106831    0
8106832    0
8106833    0
8106834    0
8106835    0
8106836    0
8106837    0
8106838    0
8106839    0
8106840    0
8106841    0
8106842    0
8106843    0
8106844    0
8106845    0
8106846    0
8106847    0
8106848    0
8106849    0
8106850    0
8106851    0
8106852    0
8106853    0
8106854    0
8106855    0
8106856    0
8106857    0
8106858    0
Name: state, Length: 1056756, dtype: int8

In [57]:
print(len(dataset['state'].unique()))
dataset['state'].unique()

16


array([ 9,  1,  0,  6, 15, 10, 14,  7,  8, 13,  3, 12,  2,  5,  4, 11])

In [58]:
# create index for each station, keep only stations with all 575 days
dataset.shape
station_indices = dataset['station'].value_counts() == 575
full_series = pd.DataFrame()
full_series['T/F'] = station_indices[station_indices]
full_series['index'] = full_series.index
dataset_fullseries = dataset[dataset['station'].isin(full_series['index'])]

In [59]:
print(dataset_fullseries.shape)
len(dataset_fullseries['station'].value_counts() == 575)

(7115050, 19)


12374

In [60]:
df = dataset_fullseries

In [61]:
# include variable that is number of days since sample started
df['date'] = pd.to_datetime(df['date'])
start = pd.to_datetime('2014-05-16')
df['num_days'] = df.apply(lambda row: (row['date'] - start).days, axis=1)

# set date as index
df.set_index('date', inplace=True)
df.sort_index(inplace=True)
cols = list(df)
cols.insert(0, cols.pop(cols.index('e5gas')))
df = df.loc[:, cols]
print(df.shape)

df.head()

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

(7115050, 19)


,e5gas,weekday,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,rotterdam,brent,wti,eurusd,vehicles,state,station,num_days
date,,,,,,,,,,,,,,,,,,,
2014-05-16,1.518000,5,48.121899,11.45140,357.104431,0,0,0,0,0,0,0.622837,109.424042,101.258553,1.369886,26056.0,1,2589,0
2014-05-16,1.575842,5,49.548401,8.35385,666.535400,0,1,0,0,0,0,0.622837,109.424042,101.258553,1.369886,52801.0,10,10137,0
2014-05-16,1.545667,5,52.889801,10.89200,48412.878906,0,0,0,0,0,0,0.622837,109.424042,101.258553,1.369886,4247.0,7,1684,0
2014-05-16,1.562684,5,51.789799,8.32017,6269.915527,0,0,0,0,0,0,0.622837,109.424042,101.258553,1.369886,3124.0,9,10049,0
2014-05-16,1.535111,5,48.793800,9.18221,7519.660156,0,0,1,0,0,0,0.622837,109.424042,101.258553,1.369886,55631.0,0,12704,0


In [62]:
# put in order to be made into supervised problem
df.sort_values(by=['station', 'num_days'], inplace=True)
df.head()

,e5gas,weekday,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,rotterdam,brent,wti,eurusd,vehicles,state,station,num_days
date,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,5,51.157501,10.0002,14850.392578,0,0,0,0,0,0,0.622837,109.424042,101.258553,1.369886,15211.0,6,0,0
2014-05-17,1.564000,6,51.157501,10.0002,14850.392578,0,0,0,0,0,0,0.622486,109.699997,101.669998,1.369580,9146.0,6,0,1
2014-05-18,1.569417,0,51.157501,10.0002,14850.392578,0,0,0,0,0,0,0.622136,109.699997,101.669998,1.369580,9843.0,6,0,2
2014-05-19,1.578167,1,51.157501,10.0002,14850.392578,0,0,0,0,0,0,0.621785,109.838928,102.008537,1.370161,13502.0,6,0,3
2014-05-20,1.599000,2,51.157501,10.0002,14850.392578,0,0,0,0,0,0,0.620475,109.522926,102.058907,1.368282,12876.0,6,0,4


In [63]:
# change the weekday and state categorical variable to be dummy variables
df = pd.get_dummies(df, drop_first=True, columns=['weekday'])
df = pd.get_dummies(df, drop_first=True, columns=['state'])
df.head()

,e5gas,latitude,longitude,dautobahn,autobahn,aral,esso,jet,shell,total,...,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15
date,,,,,,,,,,,,,,,,,,,,,
2014-05-16,1.536647,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-17,1.564000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-18,1.569417,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-19,1.578167,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2014-05-20,1.599000,51.157501,10.0002,14850.392578,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [64]:
df.to_csv("changes.csv")